In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
import xgboost as xgb

from imblearn.over_sampling import SMOTE

In [2]:
data = pd.read_csv("../input/android-permission-dataset/Android_Permission.csv")

In [3]:
data

,App,Package,Category,Description,Rating,Number of ratings,Price,Related apps,Dangerous permissions count,Safe permissions count,...,Your personal information : read calendar events (D),Your personal information : read contact data (D),Your personal information : read sensitive log data (D),Your personal information : read user defined dictionary (D),Your personal information : retrieve system internal state (S),Your personal information : set alarm in alarm clock (S),Your personal information : write Browser's history and bookmarks (D),Your personal information : write contact data (D),Your personal information : write to user defined dictionary (S),Class
0,Canada Post Corporation,com.canadapost.android,Business,Canada Post Mobile App gives you access to som...,3.1,77,0.00,"{com.adaffix.pub.ca.android, com.kevinquan.gas...",7.0,1,...,0,1,0,0,0,0,0,1,0,0
1,Word Farm,com.realcasualgames.words,Brain & Puzzle,Speed and strategy combine in this exciting wo...,4.3,199,0.00,"{air.com.zubawing.FastWordLite, com.joybits.do...",3.0,2,...,0,0,0,0,0,0,0,0,0,0
2,Fortunes of War FREE,fortunesofwar.free,Cards & Casino,"Fortunes of War is a fast-paced, easy to learn...",4.1,243,0.00,"{com.kevinquan.condado, hu.monsta.pazaak, net....",1.0,1,...,0,0,0,0,0,0,0,0,0,0
3,Better Keyboard: Avatar Purple,com.cc.betterkeyboard.skins.avatarpurple,Libraries & Demo,Skin for Better Keyboard featuring a glossy fe...,3.6,2589,0.00,{eu.gdumoulin.betterandroid.skins.transparent....,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,Boxing Day,indiaNIC.android.BoxingDay,Lifestyle,Boxing Day by Christopher Jaymes<p>Based on a ...,0.0,0,5.99,NaN,1.0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29994,Beechwood Hotel,smartstay.beechwood,Travel & Local,"Beechwood is an Android hotel app for guests, ...",5.0,2,0.00,"{com.bluewaterpages, com.travelconnection.luxu...",3.0,1,...,0,0,0,0,0,0,0,0,0,1
29995,ZDefense: Tower Defense,mobi.zstudio.avi.full,Arcade & Action,Explosive Tower Defense!<p>Top Tower Defense G...,4.6,56,2.99,"{com.entwicklerx.swampdefense, com.digimentgam...",3.0,3,...,0,0,0,0,0,0,0,0,0,1
29996,Super Task Killer FREE,com.netqin.aotkiller,Tools,Get it for FREE. Quickly Kill running apps. Fr...,4.2,6101,0.00,"{com.guardam.TaskKiller, com.netqin.authtoken_...",4.0,3,...,0,0,0,0,0,0,0,0,0,1
29997,Ambassador Hotel Taipei,VMC.Ambassadorhotel,Travel & Local,Key features of this App: <br>1. View photos o...,3.4,5,0.00,"{com.hyxen.app.NewTaipeiTravel, j2ab.android.t...",3.0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
pd.set_option('max_columns',None)
for col in data.columns:
    if data[col].isna().sum() > 0:
        print(col,data[col].isna().sum())

App 1
Description 3
Related apps 755
Dangerous permissions count 204


In [5]:
for col in data.columns:
    if data[col].dtypes == 'object':
        print(col, len(data[col].unique()))

App 22824
Package 23485
Category 30
Description 23553
Related apps 23869


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29999 entries, 0 to 29998
Columns: 184 entries, App to Class
dtypes: float64(3), int64(176), object(5)
memory usage: 42.1+ MB


In [7]:
data.describe()

,Rating,Number of ratings,Price,Dangerous permissions count,Safe permissions count,Default : Access DRM content. (S),Default : Access Email provider data (S),Default : Access all system downloads (S),Default : Access download manager. (S),Default : Advanced download manager functions. (S),Default : Audio File Access (S),Default : Install DRM content. (S),Default : Modify Google service configuration (S),Default : Modify Google settings (S),Default : Move application resources (S),Default : Read Google settings (S),Default : Send download notifications. (S),Default : Voice Search Shortcuts (S),Default : access SurfaceFlinger (S),Default : access checkin properties (S),Default : access the cache filesystem (S),Default : access to passwords for Google accounts (S),Default : act as an account authenticator (S),Default : bind to a wallpaper (S),Default : bind to an input method (S),Default : change screen orientation (S),Default : coarse (network-based) location (S),Default : control location update notifications (S),Default : control system backup and restore (S),Default : delete applications (S),Default : delete other applications' caches (S),Default : delete other applications' data (S),Default : directly call any phone numbers (S),Default : directly install applications (S),Default : disable or modify status bar (S),Default : discover known accounts (S),Default : display unauthorized windows (S),Default : enable or disable application components (S),Default : force application to close (S),Default : force device reboot (S),Default : full Internet access (S),Default : interact with a device admin (S),Default : manage application tokens (S),Default : mock location sources for testing (S),Default : modify battery statistics (S),Default : modify secure system settings (S),Default : modify the Google services map (S),Default : modify/delete USB storage contents modify/delete SD card contents (S),Default : monitor and control all application launching (S),Default : partial shutdown (S),Default : permanently disable device (S),Default : permission to install a location provider (S),Default : power device on or off (S),Default : press keys and control buttons (S),Default : prevent app switches (S),Default : read frame buffer (S),Default : read instant messages (S),Default : read phone state and identity (S),Default : record what you type and actions you take (S),Default : reset system to factory defaults (S),Default : run in factory test mode (S),Default : set time (S),Default : set wallpaper size hints (S),Default : start IM service (S),Default : update component usage statistics (S),Default : write contact data (S),Default : write instant messages (S),Development tools : enable application debugging (D),Development tools : limit number of running processes (D),Development tools : make all background applications close (D),Development tools : send Linux signals to applications (D),Hardware controls : change your audio settings (D),Hardware controls : control flashlight (S),Hardware controls : control vibrator (S),Hardware controls : record audio (D),Hardware controls : take pictures and videos (D),Hardware controls : test hardware (S),Network communication : Broadcast data messages to applications. (S),Network communication : control Near Field Communication (D),Network communication : create Bluetooth connections (D),Network communication : download files without notification (S),Network communication : full Internet access (D),Network communication : make/receive Internet calls (D),Network communication : receive data from Internet (S),Network communication : view Wi-Fi state (S),Network communication : view network state (S),Phone calls : intercept outgoing calls (D),Phone calls : modify phone state (S),Phone calls : read phone state and identity (D),Services that cost you money : directly call phone numbers (D),Services that cost you money : send SMS messages (D),Storage : modify/delete USB storage contents modify/delete SD card c

In [8]:
dropper = []
for col in data.columns[10:]:
    if (data[col].value_counts()[0] == 29999 or data[col].value_counts()[1] < 1500):
        dropper.append(col)
print(len(dropper))

158


In [9]:
def preprocess_inputs(df,lst):
    df = df.copy()
    df = df.drop_duplicates()
    df = df.drop(['App','Package','Description','Related apps'],axis=1)
    for i in lst:
        df = df.drop(i,axis=1)
    df = df.dropna(axis=0)
    encoder = LabelEncoder()
    df['Category'] = encoder.fit_transform(df['Category'])
    
    y = df['Class']
    X = df.drop('Class',axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.7, shuffle=True, random_state=43)
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), index = X_train.index, columns = X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns = X_test.columns)
    
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = preprocess_inputs(data,lst=dropper)
X_train

,Category,Rating,Number of ratings,Price,Dangerous permissions count,Safe permissions count,Hardware controls : control vibrator (S),Hardware controls : take pictures and videos (D),Network communication : full Internet access (D),Network communication : view Wi-Fi state (S),Network communication : view network state (S),Phone calls : read phone state and identity (D),Services that cost you money : directly call phone numbers (D),Storage : modify/delete USB storage contents modify/delete SD card contents (D),System tools : automatically start at boot (S),System tools : prevent device from sleeping (D),System tools : retrieve running applications (D),Your accounts : discover known accounts (S),Your location : coarse (network-based) location (D),Your location : fine (GPS) location (D),Your personal information : read contact data (D)
11025,0.135207,0.681694,-0.097652,-0.184891,0.660573,0.464654,-0.528334,3.688924,0.483373,-0.393025,0.888546,-0.751943,-0.296351,1.253644,-0.317226,-0.496358,-0.229314,4.092819,1.978583,1.937397,-0.304276
11355,-0.427665,0.681694,-0.124847,-0.184891,-0.673154,-0.206627,-0.528334,-0.271082,0.483373,-0.393025,0.888546,-0.751943,-0.296351,-0.797675,-0.317226,-0.496358,-0.229314,-0.244330,-0.505412,-0.516156,-0.304276
24133,-0.315090,0.049269,-0.111225,-0.184891,0.327141,0.464654,-0.528334,-0.271082,0.483373,2.544368,0.888546,1.329888,-0.296351,1.253644,-0.317226,2.014676,-0.229314,-0.244330,-0.505412,-0.516156,-0.304276
20754,-0.089942,0.681694,1.092117,-0.184891,2.994595,2.478496,-0.528334,-0.271082,0.483373,-0.393025,0.888546,1.329888,-0.296351,-0.797675,3.152329,2.014676,-0.229314,4.092819,1.978583,1.937397,-0.304276
7587,-1.328258,-1.145310,-0.124058,-0.184891,-0.673154,-0.877907,-0.528334,-0.271082,0.483373,-0.393025,-1.125435,-0.751943,-0.296351,-0.797675,-0.317226,-0.496358,-0.229314,-0.244330,-0.505412,-0.516156,-0.304276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29752,-0.877961,-0.021000,-0.114690,-0.184891,1.660868,1.807215,1.892741,3.688924,0.483373,2.544368,0.888546,1.329888,-0.296351,1.253644,-0.317226,-0.496358,-0.229314,4.092819,1.978583,1.937397,3.286487
8262,0.923226,0.049269,-0.124656,-0.184891,2.327731,-0.206627,-0.528334,3.688924,0.483373,-0.393025,0.888546,1.329888,3.374377,1.253644,-0.317226,-0.496358,-0.229314,-0.244330,1.978583,1.937397,-0.304276
20114,-1.328258,0.611424,0.252729,1.270872,0.994005,0.464654,-0.528334,-0.271082,0.483373,2.544368,0.888546,1.329888,-0.296351,1.253644,-0.317226,-0.496358,4.360835,-0.244330,-0.505412,-0.516156,-0.304276
21347,-0.202516,0.189808,-0.124608,-0.184891,-0.339722,-0.877907,-0.528334,-0.271082,0.483373,-0.393025,-1.125435,1.329888,-0.296351,-0.797675,-0.317226,-0.496358,-0.229314,-0.244330,-0.505412,-0.516156,-0.304276


In [10]:
smote = SMOTE(random_state=23)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
models = {
    '  Logistic Regression': LogisticRegression(),
    '           Linear SVM': LinearSVC(),
    '        XGBClassifier': xgb.XGBClassifier(),
    '    Gradient Boosting': GradientBoostingClassifier(),
    '        Decision Tree': DecisionTreeClassifier(),
    '        Random Forest': RandomForestClassifier(),
    ' KNeighborsClassifier': KNeighborsClassifier(),
    '   Bagging Classifier': BaggingClassifier()
}

for name, model in models.items():
    model = model.fit(X_train_smote, y_train_smote)
    print(name + " trained")

  Logistic Regression trained


/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


           Linear SVM trained
[16:51:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
        XGBClassifier trained
    Gradient Boosting trained
        Decision Tree trained
        Random Forest trained
 KNeighborsClassifier trained
   Bagging Classifier trained


In [11]:
for name, model in models.items():
    print(name + ": {:.2f}%".format(model.score(X_test, y_test) * 100))

  Logistic Regression: 70.61%
           Linear SVM: 70.61%
        XGBClassifier: 80.90%
    Gradient Boosting: 80.04%
        Decision Tree: 74.55%
        Random Forest: 78.10%
 KNeighborsClassifier: 71.46%
   Bagging Classifier: 77.74%


In [12]:
import tensorflow as tf
inputs = tf.keras.Input(shape=(X_train_smote.shape[1],))

x = tf.keras.layers.Dense(128, activation = 'relu')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(64, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.6)(x)
x = tf.keras.layers.Dense(32, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)


model = tf.keras.Model(inputs = inputs, outputs = outputs)

model.compile(optimizer="adam",
             loss="binary_crossentropy",
             metrics=["binary_accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               2816      
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0     

2022-03-20 16:51:42.660834: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [13]:
epochs=100
history = model.fit(
  X_train_smote,
  y_train_smote,
  validation_split=0.2,
  epochs=epochs,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=3,
            restore_best_weights=True
            )
    ]
)

2022-03-20 16:51:42.953353: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
631/631 [==============================] - 4s 5ms/step - loss: 0.5980 - binary_accuracy: 0.6533 - val_loss: 0.7028 - val_binary_accuracy: 0.6090
Epoch 2/100
631/631 [==============================] - 3s 4ms/step - loss: 0.5291 - binary_accuracy: 0.6901 - val_loss: 0.6628 - val_binary_accuracy: 0.7908
Epoch 3/100
631/631 [==============================] - 3s 4ms/step - loss: 0.5059 - binary_accuracy: 0.7083 - val_loss: 0.6781 - val_binary_accuracy: 0.7748
Epoch 4/100
631/631 [==============================] - 3s 4ms/step - loss: 0.4965 - binary_accuracy: 0.7176 - val_loss: 0.6349 - val_binary_accuracy: 0.8695
Epoch 5/100
631/631 [==============================] - 3s 4ms/step - loss: 0.4898 - binary_accuracy: 0.7215 - val_loss: 0.6099 - val_binary_accuracy: 0.9105
Epoch 6/100
631/631 [==============================] - 3s 4ms/step - loss: 0.4850 - binary_accuracy: 0.7264 - val_loss: 0.6086 - val_binary_accuracy: 0.9116
Epoch 7/100
631/631 [==============================] - 3s 

In [14]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'My test loss is {loss*100:.2f}% and test accuracy is {accuracy*100:.2f}%')

255/255 [==============================] - 1s 2ms/step - loss: 0.4653 - binary_accuracy: 0.7220
My test loss is 46.53% and test accuracy is 72.20%
